# 1D Variability analysis for HBEC IFN experiment

In [1]:
import scanpy as sc
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from pybedtools import BedTool
import pickle as pkl
%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
/data/home/anaconda3/envs/single_cell/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import sys
sys.path.append('/data/home/Github/scrna-parameter-estimation/dist/memento-0.0.4-py3.7.egg')
sys.path.append('/data/home/Github/misc-seq/miscseq/')
import encode
import memento

In [3]:
data_path = '/data_volume/ifn_hbec/'

### Read the processed RNA data

Focus on the club and bc/club cells and type I interferons for now.

Encode the timestamps to integers.

In [4]:
adata = sc.read(data_path + 'HBEC_type_I_filtered_counts_deep.h5ad')

In [5]:
adata = adata[:, ~adata.var.index.str.startswith('MT-')].copy()
adata.obs['cell_type'] = adata.obs['cell_type'].apply(lambda x: x if x != 'basal/club' else 'bc')
adata.obs['cell_type'] = adata.obs['cell_type'].apply(lambda x: x if x != 'ionocyte/tuft' else 'ion-tuft')

In [6]:
adata.shape

(69958, 36588)

### Setup memento

In [7]:
def assign_q(batch):
    
    if batch == 0:
        return 0.387*0.25
    elif batch == 1:
        return 0.392*0.25
    elif batch == 2:
        return 0.436*0.25
    else:
        return 0.417*0.25

In [8]:
adata.obs['q'] = adata.obs['batch'].apply(assign_q)

In [9]:
memento.setup_memento(adata, q_column='q')

In [10]:
adata.obs.head(3)

,NUM.SNPS,BEST.GUESS,DROPLET.TYPE,batch,HTO_classification,condition,donor,stim,time,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,leiden,cell_type,q,memento_size_factor
AAACCCAAGGAAGTAG-1,1700,donor0,SNG,0,hash-10,d2513_lambda_9,d2513,lambda,9,4297,4297,12191.0,1394.0,11.434665,3,ciliated,0.10425,0.955289
AAACCCAAGGGACTGT-1,839,donor0,SNG,0,hash-3,d2513_alpha_24,d2513,alpha,24,2171,2171,7270.0,386.0,5.309491,5,bc,0.10425,0.824407
AAACCCACAATCTGCA-1,1130,donor1,SNG,0,hash-1,d2614_alpha_3,d2614,alpha,3,3016,3016,9544.0,516.0,5.406538,0,basal,0.10425,0.988939


### Run memento for each subset, comparing to control

In [15]:
cts = [['ciliated'], ['bc', 'basal']]
tps = ['3', '6', '9', '24', '48']
stims = ['alpha', 'beta', 'gamma', 'lambda']

In [16]:
import os
done_files = os.listdir('/data_volume/ifn_hbec/binary_test_deep/')

In [17]:
for ct in cts:
    for tp in tps:
        for stim in stims:
            
            fname = '{}_{}_{}.h5ad'.format('-'.join(ct), stim, tp)
            
            if fname in done_files:
                print('Skipping', fname)
                continue

            print('starting', ct, tp, stim)

            adata_stim = adata.copy()[
                adata.obs.cell_type.isin(ct) & \
                adata.obs.stim.isin(['control', stim]) & \
                adata.obs.time.isin(['0',tp]), :].copy()
            time_converter={0:0, int(tp):1}
            adata_stim.obs['time_step'] = adata_stim.obs['time'].astype(int).apply(lambda x: time_converter[x])

            memento.create_groups(adata_stim, label_columns=['time_step', 'donor'])
            memento.compute_1d_moments(adata_stim, min_perc_group=.9)

            memento.ht_1d_moments(
                adata_stim, 
                formula_like='1 + time_step',
                cov_column='time_step', 
                num_boot=10000, 
                verbose=1,
                num_cpus=14)

            del adata_stim.uns['memento']['mv_regressor']

            adata_stim.write('/data_volume/ifn_hbec/binary_test_deep/{}_{}_{}.h5ad'.format(
                '-'.join(ct), stim, tp))

Skipping ciliated_alpha_3.h5ad
Skipping ciliated_beta_3.h5ad
Skipping ciliated_gamma_3.h5ad
Skipping ciliated_lambda_3.h5ad
Skipping ciliated_alpha_6.h5ad
Skipping ciliated_beta_6.h5ad
Skipping ciliated_gamma_6.h5ad
Skipping ciliated_lambda_6.h5ad
Skipping ciliated_alpha_9.h5ad
Skipping ciliated_beta_9.h5ad
Skipping ciliated_gamma_9.h5ad
Skipping ciliated_lambda_9.h5ad
Skipping ciliated_alpha_24.h5ad
Skipping ciliated_beta_24.h5ad
Skipping ciliated_gamma_24.h5ad
Skipping ciliated_lambda_24.h5ad
Skipping ciliated_alpha_48.h5ad
Skipping ciliated_beta_48.h5ad
Skipping ciliated_gamma_48.h5ad
Skipping ciliated_lambda_48.h5ad
starting ['bc', 'basal'] 3 alpha


[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done  22 tasks      | elapsed:    1.2s
[Parallel(n_jobs=14)]: Done 172 tasks      | elapsed:    6.3s
[Parallel(n_jobs=14)]: Done 422 tasks      | elapsed:   15.0s
[Parallel(n_jobs=14)]: Done 772 tasks      | elapsed:   27.4s
[Parallel(n_jobs=14)]: Done 1222 tasks      | elapsed:   41.9s
[Parallel(n_jobs=14)]: Done 1772 tasks      | elapsed:   59.6s
[Parallel(n_jobs=14)]: Done 2422 tasks      | elapsed:  1.4min
[Parallel(n_jobs=14)]: Done 3172 tasks      | elapsed:  1.8min
[Parallel(n_jobs=14)]: Done 4022 tasks      | elapsed:  2.2min
[Parallel(n_jobs=14)]: Done 4972 tasks      | elapsed:  2.8min
[Parallel(n_jobs=14)]: Done 6022 tasks      | elapsed:  3.4min
[Parallel(n_jobs=14)]: Done 7172 tasks      | elapsed:  4.0min
[Parallel(n_jobs=14)]: Done 8264 out of 8264 | elapsed:  4.6min finished
... storing 'memento_group' as categorical


starting ['bc', 'basal'] 3 beta


[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done  22 tasks      | elapsed:    1.0s
[Parallel(n_jobs=14)]: Done 172 tasks      | elapsed:    6.1s
[Parallel(n_jobs=14)]: Done 422 tasks      | elapsed:   15.1s
[Parallel(n_jobs=14)]: Done 772 tasks      | elapsed:   27.9s
[Parallel(n_jobs=14)]: Done 1222 tasks      | elapsed:   43.2s
[Parallel(n_jobs=14)]: Done 1772 tasks      | elapsed:  1.0min
[Parallel(n_jobs=14)]: Done 2422 tasks      | elapsed:  1.4min
[Parallel(n_jobs=14)]: Done 3172 tasks      | elapsed:  1.9min
[Parallel(n_jobs=14)]: Done 4022 tasks      | elapsed:  2.3min
[Parallel(n_jobs=14)]: Done 4972 tasks      | elapsed:  2.9min
[Parallel(n_jobs=14)]: Done 6022 tasks      | elapsed:  3.5min
[Parallel(n_jobs=14)]: Done 7172 tasks      | elapsed:  4.2min
[Parallel(n_jobs=14)]: Done 7942 out of 7942 | elapsed:  4.7min finished
... storing 'memento_group' as categorical


starting ['bc', 'basal'] 3 gamma


...

[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done  22 tasks      | elapsed:    1.1s
[Parallel(n_jobs=14)]: Done 172 tasks      | elapsed:    7.0s
[Parallel(n_jobs=14)]: Done 422 tasks      | elapsed:   17.0s
[Parallel(n_jobs=14)]: Done 772 tasks      | elapsed:   31.1s
[Parallel(n_jobs=14)]: Done 1222 tasks      | elapsed:   48.3s
[Parallel(n_jobs=14)]: Done 1772 tasks      | elapsed:  1.2min
[Parallel(n_jobs=14)]: Done 2422 tasks      | elapsed:  1.6min
[Parallel(n_jobs=14)]: Done 3172 tasks      | elapsed:  2.1min
[Parallel(n_jobs=14)]: Done 4022 tasks      | elapsed:  2.6min
[Parallel(n_jobs=14)]: Done 4972 tasks      | elapsed:  3.3min
[Parallel(n_jobs=14)]: Done 6022 tasks      | elapsed:  3.9min
[Parallel(n_jobs=14)]: Done 7172 tasks      | elapsed:  4.7min
[Parallel(n_jobs=14)]: Done 7858 out of 7858 | elapsed:  5.1min finished
... storing 'memento_group' as categorical


starting ['bc', 'basal'] 48 gamma


[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done  22 tasks      | elapsed:    0.9s
[Parallel(n_jobs=14)]: Done 172 tasks      | elapsed:    5.2s
[Parallel(n_jobs=14)]: Done 422 tasks      | elapsed:   12.7s
[Parallel(n_jobs=14)]: Done 772 tasks      | elapsed:   23.7s
[Parallel(n_jobs=14)]: Done 1222 tasks      | elapsed:   36.4s
[Parallel(n_jobs=14)]: Done 1772 tasks      | elapsed:   52.3s
[Parallel(n_jobs=14)]: Done 2422 tasks      | elapsed:  1.2min
[Parallel(n_jobs=14)]: Done 3172 tasks      | elapsed:  1.6min
[Parallel(n_jobs=14)]: Done 4022 tasks      | elapsed:  2.0min
[Parallel(n_jobs=14)]: Done 4972 tasks      | elapsed:  2.5min
[Parallel(n_jobs=14)]: Done 6022 tasks      | elapsed:  3.0min
[Parallel(n_jobs=14)]: Done 7172 tasks      | elapsed:  3.6min
[Parallel(n_jobs=14)]: Done 8422 tasks      | elapsed:  4.2min
[Parallel(n_jobs=14)]: Done 8453 out of 8453 | elapsed:  4.2min finished
... storing 'memento_group' as cate

starting ['bc', 'basal'] 48 lambda


[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done  22 tasks      | elapsed:    1.1s
[Parallel(n_jobs=14)]: Done 172 tasks      | elapsed:    6.2s
[Parallel(n_jobs=14)]: Done 422 tasks      | elapsed:   15.2s
[Parallel(n_jobs=14)]: Done 772 tasks      | elapsed:   28.2s
[Parallel(n_jobs=14)]: Done 1222 tasks      | elapsed:   43.7s
[Parallel(n_jobs=14)]: Done 1772 tasks      | elapsed:  1.0min
[Parallel(n_jobs=14)]: Done 2422 tasks      | elapsed:  1.4min
[Parallel(n_jobs=14)]: Done 3172 tasks      | elapsed:  1.9min
[Parallel(n_jobs=14)]: Done 4022 tasks      | elapsed:  2.4min
[Parallel(n_jobs=14)]: Done 4972 tasks      | elapsed:  2.9min
[Parallel(n_jobs=14)]: Done 6022 tasks      | elapsed:  3.6min
[Parallel(n_jobs=14)]: Done 7172 tasks      | elapsed:  4.2min
[Parallel(n_jobs=14)]: Done 8214 out of 8214 | elapsed:  4.9min finished
... storing 'memento_group' as categorical
